In [1]:
import numpy as np

In [2]:
# Загрузить и подготовить тренировочные данные из формата CSV в список
training_data = open("dataset/Data_train.csv", 'r') # 'r' - открываем файл для чтения
training_data_list = training_data.readlines() # readlines() - читает все строки в файле в переменную training_data_list
training_data.close() # закрываем файл csv

In [3]:
# Определение класса нейронной сети
class neuron_Net:
    
    # Инициализация весов нейронной сети
    def __init__(self, input_num, neuron_num, output_num, learningrate):
                                         # МАТРИЦА ВЕСОВ
        # Задаем матрицу весов как случайное
        self.weights = (np.random.rand(neuron_num, input_num) +0.0)
        self.weights_out = (np.random.rand(output_num, neuron_num) +0.0)
        
        # Задаем параметр скорости обучения
        self.lr = learningrate
        
        pass
    
    # Метод обучения нейронной сети
    def train(self, inputs_list, targets_list): # принимает (вх. список данных, ответы)
        # Преобразовать список входов в вертикальный массив. .T - транспонирование
        inputs_x = np.array(inputs_list, ndmin=2).T # матрица числа
        targets_Y = np.array(targets_list, ndmin=2).T # матрица ответов: какое это число
        
                                           # ВЫЧИСЛЕНИЕ СИГНАЛОВ
        # Вычислить сигналы в нейронах скрытого слоя. Взвешенная сумма.
        x1 = np.dot(self.weights, inputs_x) # dot - умножение матриц X = W*I = weights * inputs
        # Вычислить сигналы, выходящие из нейрона. Функция активации - сигмоида(x)
        y1 = 1/(1+np.exp(-x1))
        # Вычислить сигналы в нейронах выходного слоя. Взвешенная сумма.
        x2 = np.dot(self.weights_out, y1) # dot - умножение матриц X = W*I = weights * inputs
        # Вычислить сигналы, выходящие из нейрона. Функция активации - сигмоида(x)
        y2 = 1/(1+np.exp(-x2))
        
                                            # ВЫЧИСЛЕНИЕ ОШИБКИ
        #  Ошибка E = -(цель - фактическое значение) 
        E = -(targets_Y - y2)
        # Ошибка скрытого слоя
        E_hidden = np.dot(self.weights_out.T, E) 
        
                                            # ОБНОВЛЕНИЕ ВЕСОВ
        # Меняем веса по каждой связи
        self.weights_out -= self.lr * np.dot((E * y2 * (1.0 - y2)), np.transpose(y1))
        # Меняем веса по каждой связи
        self.weights -= self.lr * np.dot((E_hidden * y1 * (1.0 - y1)), np.transpose(inputs_x))
        
        pass
    
    # Метод прогона тестовых значений
    def query(self, inputs_list): # Принимает свой набор тестовых данных
        # Преобразовать список входов в вертикальный массив. 
        inputs_x = np.array(inputs_list, ndmin=2).T 
        
        # Вычислить сигналы в нейронах скрытого слоя. Взвешенная сумма.
        x1 = np.dot(self.weights, inputs_x) # dot - умножение матриц X = W*I = weights * inputs
        # Вычислить сигналы, выходящие из нейрона. Функция активации - сигмоида(x)
        y1 = 1/(1+np.exp(-x1))
        # Вычислить сигналы в нейронах выходного слоя. Взвешенная сумма.
        x2 = np.dot(self.weights_out, y1) # dot - умножение матриц X = W*I = weights * inputs
        # Вычислить сигналы, выходящие из нейрона. Функция активации - сигмоида(x)
        y2 = 1/(1+np.exp(-x2))
        
        return y2

In [4]:
                            # ЗАДАЁМ ПАРАМЕТРЫ СЕТИ
# Количество входных данных, нейронов
data_input = 2
data_neuron = 4
data_output = 2

# Cкорость обучения
learningrate = 0.2

# Создать экземпляр нейронной сети
n = neuron_Net(data_input, data_neuron, data_output, learningrate)

In [5]:
                            # ОБУЧЕНИЕ
# Зададим количество эпох
epochs = 80000
# Прогон по обучающей выборке
for e in range(epochs):
    for i in training_data_list:
        # Получить входные данные числа
        all_values = i.split(',') # split(',') - раздел строку на символы где запятая "," символ разделения
        inputs_x = np.asfarray(all_values[1:])
        
        # Получить целевое значение Y, (ответ - какое это число)
        targets_Y = int(all_values[0])  # перевод символов в int, 0 элемент - ответ
        
        # создать целевые выходные значения (все 0.01, кроме нужной метки, которая равна 0.99)
        targets_Y = np.zeros(data_output) + 0.01
        
        # Получить целевое значение Y, (ответ - какое это число). all_values[0] - целевая метка для этой записи
        targets_Y[int(all_values[0])] = 0.99
            
        n.train(inputs_x, targets_Y) # наш метод train - обучение нейронной сети                      

In [6]:
# Вывод обученных весов
print('Весовые коэффициенты:\n', n.weights)

Весовые коэффициенты:
 [[ 12.95040394  -6.49232526]
 [ -9.08125352  -8.94676728]
 [-13.09373108   6.56168093]
 [  7.32763501 -14.51411564]]


In [7]:
# Прогоним входные данные из обучающей выборки через обученную сеть
for i in training_data_list:
    all_values = i.split(',') # split(',') - раздел строку на символы где запятая "," символ разделения
    #all_values = np.asfarray(all_values,int) # Перевод списка в int 
    inputs_x = np.asfarray(all_values[1:])
    # Прогон по сети
    outputs = n.query(inputs_x)
    print(int(all_values[1]), 'XOR', int(all_values[2]), '=' ,  np.argmax(outputs), '\n',outputs)      

0 XOR 0 = 0 
 [[ 0.97987662]
 [ 0.02012364]]
1 XOR 0 = 1 
 [[ 0.0181055 ]
 [ 0.98189458]]
0 XOR 1 = 1 
 [[ 0.01725169]
 [ 0.98274808]]
1 XOR 1 = 0 
 [[ 0.985501 ]
 [ 0.0144984]]
